In [2]:
pip install xgboost

Looking in indexes: https://mirrors.aliyun.com/pypi/simple
     |████████████████████████████████| 166.7 MB 102 kB/s eta 0:00:012     |██████████████████████████▎     | 137.2 MB 17.0 MB/s eta 0:00:02     |███████████████████████████▊    | 144.2 MB 4.6 MB/s eta 0:00:05
Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install gensim

Looking in indexes: https://mirrors.aliyun.com/pypi/simple
     |████████████████████████████████| 23.9 MB 7.3 MB/s eta 0:00:01     |██████████████████████▌         | 16.8 MB 371 kB/s eta 0:00:20
     |████████████████████████████████| 58 kB 43 kB/s eta 0:00:0101
Note: you may need to restart the kernel to use updated packages.


In [5]:
pip install python-Levenshtein

Looking in indexes: https://mirrors.aliyun.com/pypi/simple
Note: you may need to restart the kernel to use updated packages.


In [6]:
import pandas as pd
import time
TRAIN_PATH = "dataset/yelp_review_full/train.csv"
TEST_PATH = "dataset/yelp_review_full/test.csv"
df_train = pd.read_csv(TRAIN_PATH, header=None)
df_test = pd.read_csv(TEST_PATH, header=None)

# Bayes

In [3]:
t = time.perf_counter()
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(token_pattern='\[?\w+\]?')
X_train = vectorizer.fit_transform(df_train[1])
y_train = df_train[0]
t = time.perf_counter()-t
print(t)

In [4]:
X_test = vectorizer.transform(df_test[1])
y_test = df_test[0]

In [5]:
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB()
clf.fit(X_train, y_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [6]:
# Predicting results with the model on the test set
y_pred = clf.predict(X_test)

In [7]:
# Test set effect test
from sklearn import metrics

print(metrics.classification_report(y_test, y_pred))
print("accuracy:", metrics.accuracy_score(y_test, y_pred))

              precision    recall  f1-score   support

           1       0.61      0.65      0.63     10000
           2       0.45      0.43      0.44     10000
           3       0.46      0.47      0.47     10000
           4       0.48      0.51      0.49     10000
           5       0.67      0.59      0.63     10000

    accuracy                           0.53     50000
   macro avg       0.53      0.53      0.53     50000
weighted avg       0.53      0.53      0.53     50000

accuracy: 0.53104


# SVM

In [11]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(token_pattern='\[?\w+\]?')
X_train = vectorizer.fit_transform(df_train[1])
y_train = df_train[0]

In [12]:
X_test = vectorizer.transform(df_test[1])
y_test = df_test[0]

In [ ]:
from sklearn import svm
t = time.perf_counter()
clf = svm.SVC(gamma='scale')
clf.fit(X_train, y_train)
t = time.perf_counter()-t
print(t)

In [ ]:
# Predict results with the model on the test set
y_pred = clf.predict(X_test)

In [ ]:
# Test set effect test
from sklearn import metrics
print(metrics.classification_report(y_test, y_pred))
print("accuracy:", metrics.accuracy_score(y_test, y_pred))

# XGboost

In [6]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(token_pattern='\[?\w+\]?', max_features=2000)
X_train = vectorizer.fit_transform(df_train[1])
y_train = df_train[0]

In [7]:
X_test = vectorizer.transform(df_test[1])
y_test = df_test[0]

In [8]:
import xgboost as xgb
param = {
    'booster':'gbtree',
    'max_depth': 6, 
    'colsample_bytree': 0.8,
    'objective': 'multi:softmax',
    'eval_metric': 'error',
    'eta': 0.3,
    'nthread': 10,
    'num_class': 6,
}
dmatrix = xgb.DMatrix(X_train, label=y_train)
model = xgb.train(param, dmatrix, num_boost_round=200)

In [9]:
# Predict results with the model on the test set
dmatrix = xgb.DMatrix(X_test)
y_pred = model.predict(dmatrix)

In [10]:
# Test set effect test
from sklearn import metrics
print(metrics.classification_report(y_test, y_pred))
print("accuracy:", metrics.accuracy_score(y_test, y_pred))

              precision    recall  f1-score   support

           1       0.68      0.74      0.71     10000
           2       0.52      0.49      0.50     10000
           3       0.52      0.48      0.50     10000
           4       0.49      0.47      0.48     10000
           5       0.64      0.69      0.67     10000

    accuracy                           0.57     50000
   macro avg       0.57      0.57      0.57     50000
weighted avg       0.57      0.57      0.57     50000

accuracy: 0.57428


# LSTM

In [7]:
# Input format required by word2vec: list(word)
wv_input = df_train[1].map(lambda s: s.split(" "))
wv_input.head()

0    [dr., goldberg, offers, everything, i, look, f...
1    [Unfortunately,, the, frustration, of, being, ...
2    [Been, going, to, Dr., Goldberg, for, over, 10...
3    [Got, a, letter, in, the, mail, last, week, th...
4    [I, don't, know, what, Dr., Goldberg, was, lik...
Name: 1, dtype: object

In [ ]:
from gensim import models
# Word2Vec
t = time.perf_counter()
word2vec = models.Word2Vec(wv_input, 
                           vector_size=64,   # Word vector dimension
                           min_count=1,      # Minimum word frequency, because of the small amount of data, here card 1
                           epochs=1000)      # Ieration rounds
t = time.perf_counter()-t
print(t)

/environment/python/versions/miniconda3-4.7.12/lib/python3.7/site-packages/gensim/similarities/__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


In [ ]:
import torch
from torch import nn
from torch.nn.utils.rnn import pad_sequence, pack_padded_sequence,pad_packed_sequence
from torch.utils.data import Dataset, DataLoader
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
# Hyperparameters
learning_rate = 5e-4
input_size = 768
num_epoches = 5
batch_size = 100
embed_size = 64
hidden_size = 64
num_layers = 2

In [ ]:
# Datasets
class MyDataset(Dataset):
    def __init__(self, df):
        self.data = []
        self.label = df[0].tolist()
        for s in df[1].tolist():
            vectors = []
            for w in s.split(" "):
                if w in word2vec.wv.key_to_index:
                    vectors.append(word2vec.wv[w])
            vectors = torch.Tensor(vectors)
            self.data.append(vectors)
    
    def __getitem__(self, index):
        data = self.data[index]
        label = self.label[index]
        return data, label

    def __len__(self):
        return len(self.label)

def collate_fn(data):
    data.sort(key=lambda x: len(x[0]), reverse=True) 
    data_length = [len(sq[0]) for sq in data]
    x = [i[0] for i in data]
    y = [i[1] for i in data]
    data = pad_sequence(x, batch_first=True, padding_value=0)   # Necessary operations for processing variable-length sequences with RNN
    return data, torch.tensor(y, dtype=torch.float32), data_length


# Training set
train_data = MyDataset(df_train)
train_loader = DataLoader(train_data, batch_size=batch_size, collate_fn=collate_fn, shuffle=True)

# Test sets
test_data = MyDataset(df_test)
test_loader = DataLoader(test_data, batch_size=batch_size, collate_fn=collate_fn, shuffle=True)

In [ ]:
# Network structure
class LSTM(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers):
        super(LSTM, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True, bidirectional=True)
        self.fc = nn.Linear(hidden_size * 2, 1)  # Bidirectional, output dimension to be *2
        self.sigmoid = nn.Sigmoid()

    def forward(self, x, lengths):
        h0 = torch.zeros(self.num_layers * 2, x.size(0), self.hidden_size).to(device)  # Two-way, first dimension to be *2
        c0 = torch.zeros(self.num_layers * 2, x.size(0), self.hidden_size).to(device)
        
        packed_input = torch.nn.utils.rnn.pack_padded_sequence(input=x, lengths=lengths, batch_first=True)
        packed_out, (h_n, h_c) = self.lstm(packed_input, (h0, c0))

        lstm_out = torch.cat([h_n[-2], h_n[-1]], 1)  # Bidirectional, so the last two dimensions are stitched together to give the output of the last time step
        out = self.fc(lstm_out)
        out = self.sigmoid(out)
        return out

lstm = LSTM(embed_size, hidden_size, num_layers)

In [ ]:
from sklearn import metrics

# In test set effect test
def test():
    y_pred, y_true = [], []

    with torch.no_grad():
        for x, labels, lengths in test_loader:
            x = x.to(device)
            outputs = lstm(x, lengths)          # Forward propagation
            outputs = outputs.view(-1)          # Flattening the output
            y_pred.append(outputs)
            y_true.append(labels)

    y_prob = torch.cat(y_pred)
    y_true = torch.cat(y_true)
    y_pred = y_prob.clone()
    print(y_pred)
    print(metrics.classification_report(y_true, y_pred))
    print("accuracy:", metrics.accuracy_score(y_true, y_pred))

In [ ]:
# Define loss functions and optimisers
criterion = nn.BCELoss()
optimizer = torch.optim.Adam(lstm.parameters(), lr=learning_rate)

In [ ]:
# Iterative training
t = time.perf_counter()
for epoch in range(num_epoches):
    total_loss = 0
    for i, (x, labels, lengths) in enumerate(train_loader):
        x = x.to(device)
        labels = labels.to(device)
        outputs = lstm(x, lengths)          # Forward propagation
        logits = outputs.view(-1)           # Flattening the output
        loss = criterion(logits, labels)    # loss calculation
        total_loss += loss
        optimizer.zero_grad()               # Gradient clearing
        loss.backward(retain_graph=True)    # Back propagation, calculating gradients
        optimizer.step()                    # Gradient update
        if (i+1) % 10 == 0:                                                                                                                                                            
            print("epoch:{}, step:{}, loss:{}".format(epoch+1, i+1, total_loss/10))
            total_loss = 0
    
    # test
    test()
    
    # save model
    model_path = "./model/lstm_{}.model".format(epoch+1)
    torch.save(lstm, model_path)
    print("saved model: ", model_path)
t = time.perf_counter()-t
print(t)